## GPU Setup

In [3]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
if (device_name != b'Tesla T4') and (device_name != b'Tesla P4') and (device_name != b'Tesla P100-PCIE-16GB'):
  raise Exception("Unfortunately this instance does not have a T4, P4 or P100 GPU.Please make sure you've configured Colab to request a GPU instance type.\
  Sometimes Colab allocates a Tesla K80 instead of a T4, P4 or P100. Resetting the instance.If you get a K80 GPU, try Runtime -> Reset all runtimes...")
else:
  print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


In [4]:
!nvidia-smi

Mon Sep 28 15:02:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
import pandas as pd
from tqdm.notebook import tqdm
import torch
import os

## Import Data

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Others/Second/BERT Sentiment Analysis/Kaggle 1.6M/data.csv', names=[
    'target', 'id', 'date', 'flag', 'user', 'text'], encoding='ISO-8859-1')
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Preprocessing

In [8]:
df = df.sample(frac=1, random_state=17)

In [9]:
df_0 = df[df['target']==0]
df_1 = df[df['target']==4]
df_new = pd.concat([df_0.iloc[:20000], df_1.iloc[:20000]])
df = df_new.reset_index(drop=True)

In [10]:
df.target.value_counts()

4    20000
0    20000
Name: target, dtype: int64

In [11]:
df = df.loc[:, ['target', 'text']]
df.head()

,target,text
0,0,Listening to friends talk politics. Shouldn't ...
1,0,Allies not coming over
2,0,sore throat!
3,0,Sooo....today i'll be studying computer theory...
4,0,Is Too Tired To Function


In [12]:
label_dict = {'happy':1, 'sad':0}

In [13]:
def change_value(x):
  if x==4:
    return 1
  else:
    return x

In [14]:
df['target'] = df['target'].apply(lambda x: change_value(x))
df.tail()

,target,text
39995,1,@ddlovato get well for tonight demi! i'm weari...
39996,1,@joeymcintyre do I still Have your permission ...
39997,1,@r27 thanks for all the quality links dude
39998,1,haha my mom probably doesn't know i just made ...
39999,1,@p_cook there you go mate http://www.quidco.c...


## Preprocessing

In [15]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer 

In [16]:
!pip install nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
# TEXT CLEANING
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+" #last part is used to remove special chars and punctuation


def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

In [18]:
df['text'] = df['text'].apply(lambda x: preprocess(x))

### Replace words

In [19]:
contractions = {
"aight": "alright",
"ain't": "am not",
"amn't": "am not",
"aren't": "are not",
"can't": "can not",
"cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"daren't": "dare not",
"daresn't": "dare not",
"dasn't": "dare not",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"d'ye": "do you",
"e'er": "ever",
"everybody's": "everybody is",
"everyone's": "everyone is",
"finna": "fixing to",
"g'day": "good day",
"gimme": "give me",
"giv'n": "given",
"gonna": "going to",
"gon't": "go not",
"gotta": "got to",
"hadn't": "had not",
"had've": "had have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'dn't've'd": "he would not have had",
"he'll": "he will",
"he's": "he is",
"he've": "he have",
"how'd": "how would",
"howdy": "how do you do",
"how'll": "how will",
"how're": "how are",
"I'll": "I will",
"I'm": "I am",
"I'm'a": "I am about to",
"I'm'o": "I am going to",
"innit": "is it not",
"I've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"may've": "may have",
"methinks": "me thinks",
"mightn't": "might not",
"might've": "might have",
"mustn't": "must not",
"mustn't've": "must not have",
"must've": "must have",
"needn't": "need not",
"ne'er": "never",
"o'clock": "of the clock",
"o'er": "over",
"ol'": "old",
"oughtn't": "ought not",
"'s": "is",
"shalln't": "shall not",
"shan't": "shall not",
"she'd": "she would",
"she'll": "she shall",
"she'll": "she will",
"she's": "she has",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"somebody's": "somebody has",
"somebody's": "somebody is",
"someone's": "someone has",
"someone's": "someone is",
"something's": "something has",
"something's": "something is",
"so're": "so are",
"that'll": "that shall",
"that'll": "that will",
"that're": "that are",
"that's": "that has",
"that's": "that is",
"that'd": "that would",
"that'd": "that had",
"there'd": "there had",
"there'd": "there would",
"there'll": "there shall",
"there'll": "there will",
"there're": "there are",
"there's": "there has",
"there's": "there is",
"these're": "these are",
"these've": "these have",
"they'd": "they had",
"they'd": "they would",
"they'll": "they shall",
"they'll": "they will",
"they're": "they are",
"they're": "they were",
"they've": "they have",
"this's": "this has",
"this's": "this is",
"those're": "those are",
"those've": "those have",
"'tis": "it is",
"to've": "to have",
"'twas": "it was",
"wanna": "want to",
"wasn't": "was not",
"we'd": "we had",
"we'd": "we would",
"we'd": "we did",
"we'll": "we shall",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'd": "what did",
"what'll": "what shall",
"what'll": "what will",
"what're": "what are",
"what're": "what were",
"what's": "what has",
"what's": "what is",
"what's": "what does",
"what've": "what have",
"when's": "when has",
"when's": "when is",
"where'd": "where did",
"where'll": "where shall",
"where'll": "where will",
"where're": "where are",
"where's": "where has",
"where's": "where is",
"where's": "where does",
"where've": "where have",
"which'd": "which had",
"which'd": "which would",
"which'll": "which shall",
"which'll": "which will",
"which're": "which are",
"which's": "which has",
"which's": "which is",
"which've": "which have",
"who'd": "who would",
"who'd": "who had",
"who'd": "who did",
"who'd've": "who would have",
"who'll": "who shall",
"who'll": "who will",
"who're": "who are",
"who's": "who has",
"who's": "who is",
"who's": "who does",
"who've": "who have",
"why'd": "why did",
"why're": "why are",
"why's": "why has",
"why's": "why is",
"why's": "why does",
"won't": "will not",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd've": "you all would have",
"y'all'dn't've'd": "you all would not have had",
"y'all're": "you all are",
"you'd": "you had",
"you'd": "you would",
"you'll": "you shall",
"you'll": "you will",
"you're": "you are",
"you're": "you are",
"you've": "you have",
" u ": "you",
" ur ": "your",
" n ": "and"
}

In [20]:
def cont_to_exp(x):
    if type(x) is str:
        for key in contractions:
            value = contractions[key]
            x = x.replace(key,value)
        return x
    else:
        return x

In [21]:
df['text'] = df['text'].apply(lambda x: cont_to_exp(x))

### Remove emails

In [22]:
import re

df['Emails'] = df['text'].apply(lambda x: re.findall(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)',x))

df['Emails_count'] = df['Emails'].apply(lambda x: len(x))

df['text'] = df['text'].apply(lambda x: re.sub(r'([a-zA-Z0-9+._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)', '',x))

### Remove Retweets

In [23]:
df['text'] = df['text'].apply(lambda x: re.sub('RT', '', x))

### Remove Multiple Spaces

In [24]:
df['text'] = df['text'].apply(lambda x: ' '.join(x.split()))

### Remove html tags

In [25]:
from bs4 import BeautifulSoup

df['text'] = df['text'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

### Common word removal

In [26]:
text = ' '.join(df['text'])

text = text.split()

freq_comm = pd.Series(text).value_counts()

f_20 = freq_comm[:20]

df['text'] = df['text'].apply(lambda x: " ".join([t for t in x.split() if t not in f_20]))

### Rare word removal

In [27]:
rare_20 = freq_comm[-20:]
df['text'] = df['text'].apply(lambda x: ' '.join([t for t in x.split() if t not in rare_20]))

## Train/Validation Split

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.target.values,
                                                  test_size=0.1,
                                                  random_state=17,
                                                  stratify=df.target.values)

In [30]:
df['data_type'] = ['not_set']*df.shape[0]

In [31]:
df.head()

,target,text,Emails,Emails_count,data_type
0,0,listening friends talk politics bored birthday,[],0,not_set
1,0,allies coming,[],0,not_set
2,0,sore throat,[],0,not_set
3,0,sooo studying computer theory exam tomorrow,[],0,not_set
4,0,tired function,[],0,not_set


In [32]:
X_train.shape

(36000,)

In [33]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [34]:
df.head()

,target,text,Emails,Emails_count,data_type
0,0,listening friends talk politics bored birthday,[],0,train
1,0,allies coming,[],0,train
2,0,sore throat,[],0,val
3,0,sooo studying computer theory exam tomorrow,[],0,train
4,0,tired function,[],0,train


In [35]:
df = df.loc[:, ['target', 'text', 'data_type']]

## Load Tokenizer and Encode Data

In [36]:
!pip install transformers

     |████████████████████████████████| 1.1MB 6.4MB/s 
     |████████████████████████████████| 890kB 31.8MB/s 
     |████████████████████████████████| 3.0MB 47.7MB/s 
     |████████████████████████████████| 1.1MB 47.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=6c1d94fdd08d945c8668991dc1c3d3fce6ab9a2d2ad7e6e1db657fc8b03b5b2a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [37]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [38]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased', 
    do_lower_case=True 
)

## New Tokenization

In [39]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].target.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [40]:
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].target.values)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [41]:
print(df[df.data_type=='train'].target.value_counts())
print(df[df.data_type=='val'].target.value_counts())

1    18000
0    18000
Name: target, dtype: int64
1    2000
0    2000
Name: target, dtype: int64


In [42]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [45]:
len(dataset_train)

36000

In [46]:
len(dataset_val)

4000

## Set up BERT Pretrained Model

In [47]:
from transformers import BertForSequenceClassification

In [48]:
# print(len(label_dict))
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

for param in model.bert.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Create Data Loaders

In [49]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [50]:
batch_size=32

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=batch_size
)

## Setup Optimizer and Scheduler

In [51]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [52]:
optimizer=AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8
)

In [54]:
epochs=19

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)

## Define Performance Metrics

In [55]:
import numpy as np
from sklearn.metrics import f1_score

In [56]:
def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted')

In [57]:
def func_accuracy_per_class(preds, labels):
  label_dict_inverse = {v:k for k,v in label_dict.items()}
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat==label]
    y_true = labels_flat[labels_flat==label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')

## Create Training Loop

In [58]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [59]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [60]:
def evaluate(dataloader_val):
  model.eval()

  loss_val_total = 0
  predictions, true_vals = [], []
  for batch in dataloader_val:
    batch = tuple(b.to(device) for b in batch)

    inputs = {
        'input_ids':batch[0],
        'attention_mask':batch[1],
        'labels':batch[2]
    }
    with torch.no_grad(): #no gradient change here, so no training occurs
      outputs = model(**inputs)

    loss = outputs[0]
    logits = outputs[1]
    loss_val_total += loss.item()

    logits = logits.detach().cpu().numpy()
    label_ids = inputs['labels'].cpu().numpy()
    predictions.append(logits)
    true_vals.append(label_ids)
  
  loss_val_avg = loss_val_total/len(dataloader_val) 

  predictions = np.concatenate(predictions, axis=0)
  true_vals = np.concatenate(true_vals, axis=0)

  return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
  #unfreeze layers
  if epoch == 10:
    for param in model.bert.parameters():
      param.requires_grad = True


  model.train()
  
  loss_train_total = 0

  progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch),
                      leave=False, disable=False)
  
  for batch in progress_bar:
    model.zero_grad()

    batch = tuple(b.to(device) for b in batch)

    inputs = {
        'input_ids':batch[0],
        'attention_mask':batch[1],
        'labels':batch[2]
    }
    outputs = model(**inputs)

    loss = outputs[0]
    loss_train_total += loss.item()
    loss.backward()

    #prevents vanishing or exploding and helps generalization
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step() 
    scheduler.step() 

    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

  

  #save model every epoch with all layers and weights
  if epoch > 10:
    torch.save(model.state_dict(), f'/content/gdrive/My Drive/Colab Notebooks/Others/Second/BERT Sentiment Analysis/Kaggle 1.6M/Models/BERT_ft_epoch{epoch}.model')

  tqdm.write(f'\n Epoch {epoch}')

  loss_train_avg = loss_train_total/len(dataloader_train)
  tqdm.write(f'Training Loss: {loss_train_avg}')

  val_loss, predictions, true_vals = evaluate(dataloader_val)
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f'Validation Loss: {val_loss}')
  tqdm.write(f'F1 score weighted: {val_f1}')



 Epoch 1
Training Loss: 0.6956424763997395
Validation Loss: 0.6887753686904907
F1 score weighted: 0.4756174328847235



 Epoch 2
Training Loss: 0.6903490295410156
Validation Loss: 0.6831014218330383
F1 score weighted: 0.5451803492009962



 Epoch 3
Training Loss: 0.6864603736135695
Validation Loss: 0.6792554388046265
F1 score weighted: 0.589586246150099



 Epoch 4
Training Loss: 0.6838863787121243
Validation Loss: 0.676674605846405
F1 score weighted: 0.6070751817277285



 Epoch 5
Training Loss: 0.68028108798133
Validation Loss: 0.6742580375671386
F1 score weighted: 0.5836279662366619


...


 Epoch 16
Training Loss: 0.6654133801460266
Validation Loss: 0.6569572143554687
F1 score weighted: 0.6447867832666558



 Epoch 17
Training Loss: 0.6654155224694146
Validation Loss: 0.6562899513244629
F1 score weighted: 0.6437499777343735



 Epoch 18
Training Loss: 0.6641013673146565
Validation Loss: 0.6558368062973022
F1 score weighted: 0.6420804891850512



 Epoch 19
Training Loss: 0.6636651065084669
Validation Loss: 0.6551806850433349
F1 score weighted: 0.6443355941283858


Buffered data was truncated after reaching the output size limit.

## Load and Evaluate Model

In [61]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [62]:
model.to(device)
pass

In [63]:
direc_path = "/content/gdrive/My Drive/Colab Notebooks/Others/Second/BERT Sentiment Analysis/Kaggle 1.6M/Final 1"
model.load_state_dict(torch.load(os.path.join(direc_path, 'BERT_ft_epoch10.model'), map_location=torch.device('cuda')))

<All keys matched successfully>

In [64]:
_, predictions, true_vals = evaluate(dataloader_val)

In [65]:
func_accuracy_per_class(predictions, true_vals)

Class: sad
Accuracy: 1606/2000
Class: happy
Accuracy: 1477/2000
